In [12]:
from tiingo import TiingoClient
import pandas as pd
from datetime import datetime as dt 
import datetime

In [13]:
client = TiingoClient({'api_key':'YOUR-KEY'})
start_date = dt.today() - datetime.timedelta(days=365)
end_date = str(dt.today().strftime('%Y-%m-%d'))

In [14]:
def get_tickers(tickers,start_date,end_date):
     
    ticker_dataframe = client.get_dataframe(tickers,
                                      frequency ='daily',
                                      metric_name = 'adjClose', 
                                      startDate = start_date,
                                      endDate = end_date)
    
    ticker_dataframe = ticker_dataframe.iloc[::-1]
    
    return ticker_dataframe

In [15]:
def rsc_algo_test(dataframe,timeframe):
    columns = list(dataframe)
    base_instrument = dataframe.columns.values[-1]
    sorted_symbols=[]
    
    for column in columns:
        if column == base_instrument:
            continue
        sorted_symbols.append(tuple((column,(dataframe[column][timeframe] / dataframe[base_instrument][timeframe])[::-1].pct_change().sum(),dataframe[column][0],
                                     round((dataframe[column][timeframe][::-1].pct_change().sum())*100,2),base_instrument))) # output tuple with ticker,rsc
        
        sorted_symbols.sort( key=lambda x: x[1],reverse=True)
    return sorted_symbols


In [16]:
# get the sector into the list of stocks 
def flatten(input_list):
    output_list = []
    for element in input_list:
        if type(element) == list:
            output_list.extend(flatten(element))
        else:
            output_list.append(element)
    return output_list

In [17]:
sectors = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF','XLRE','SPY']
sectors_app_stocks = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF','XLRE']

In [18]:
# update ticker holdings in sectors
sector_stock_list = ['xle','xlu','xlk','xlb','xlp','xly','xli','xlc','xlv','xlf','xlre']

In [20]:
# fetch the actual ticker holdings in the sectors and clean the list of unwanted strings
xle,xlu,xlk,xlb,xlp,xly,xli,xlc,xlv,xlf,xlre = (pd.read_csv('https://www.sectorspdr.com/sectorspdr/IDCO.Client.Spdrs.Portfolio/Export/ExportCsv?symbol='+str(x))
                                                            for x in sector_stock_list)

sector_csv_list  = [xle,xlu,xlk,xlb,xlp,xly,xli,xlc,xlv,xlf,xlre] 

xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers =(list(x.index.get_level_values(0)) 
                                                                                                                                       for x in sector_csv_list)

ticker_list = [xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers]


ticker_list_re = [xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers]

xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers = [i[1:] 
                                                                                                                                        for i in ticker_list_re]


In [21]:
stock_tickers = [xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers ]

xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers = map(list,zip(
                                                                                                                        stock_tickers,sectors_app_stocks))

stock_tickers_test = [xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers ]

xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers = [flatten(x) for x in stock_tickers_test]

stock_tickers = [xle_tickers,xlu_tickers,xlk_tickers,xlb_tickers,xlp_tickers,xly_tickers,xli_tickers,xlc_tickers,xlv_tickers,xlf_tickers,xlre_tickers ]

stock_tickers_clean = [[x for x in y if str(x) != 'nan'] for y in stock_tickers]

stock_tickers_final = [[x for x in y if x.isalpha()]for y in stock_tickers_clean]



11


In [22]:
#get stock data

xle_df,xlu_df,xlk_df,xlb_df,xlp_df,xly_df,xli_df,xlc_df,xlv_df,xlf_df,xlre_df = (get_tickers(x,start_date,end_date) for x in stock_tickers_final)

C:\ProgramData\Anaconda3\lib\site-packages\tiingo\api.py:270: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [11]:
ticker_dataframe = [xle_df,xlu_df,xlk_df,xlb_df,xlp_df,xly_df,xli_df,xlc_df,xlv_df,xlf_df,xlre_df]

In [12]:
# get sectors data

sectors_df = get_tickers(sectors,start_date,end_date)

In [13]:
daily = slice(0,2)
weekly = slice(0,6)
monthly = slice(0,31)
yearly = slice(0,366)

In [14]:
#run the rsc algo on sectors

sector_daily = rsc_algo_test(sectors_df,daily) 
sector_weekly = rsc_algo_test(sectors_df,weekly)
sector_monthly = rsc_algo_test(sectors_df,monthly) 
sector_yearly = rsc_algo_test(sectors_df,yearly)

In [15]:
# algo ouput into dataframe 

sector_daily_df =  pd.DataFrame(sector_daily,columns=['Ticker','RSC','Price','Percent Change','Index'])
sector_weekly_df =  pd.DataFrame(sector_weekly,columns=['Ticker','RSC','Price','Percent Change','Index'])
sector_monthly_df =  pd.DataFrame(sector_monthly,columns=['Ticker','RSC','Price','Percent Change','Index'])
sector_yearly_df =  pd.DataFrame(sector_yearly,columns=['Ticker','RSC','Price','Percent Change','Index'])

In [16]:
# Stock Tickers rsc algo > to dataframe

In [17]:
xle_daily,xlu_daily,xlk_daily,xlb_daily,xlp_daily,xly_daily,xli_daily,xlc_daily,xlv_daily,xlf_daily,xlre_daily = (rsc_algo_test(x,daily) for x in ticker_dataframe)

tickers_daily = [xle_daily,xlu_daily,xlk_daily,xlb_daily,xlp_daily,xly_daily,xli_daily,xlc_daily,xlv_daily,xlf_daily,xlre_daily]

xle_daily_df,xlu_daily_df,xlk_daily_df,xlb_daily_df,xlp_daily_df,xly_daily_df,xli_daily_df,xlc_daily_df,xlv_daily_df,xlf_daily_df,xlre_daily_df = (pd.DataFrame(x,columns=['Ticker','RSC','Price','Percent Change','Index'])
                                                                                                                                                   for x in tickers_daily)

In [18]:
xle_weekly,xlu_weekly,xlk_weekly,xlb_weekly,xlp_weekly,xly_weekly,xli_weekly,xlc_weekly,xlv_weekly,xlf_weekly,xlre_weekly = (rsc_algo_test(x,weekly) for x in ticker_dataframe)

tickers_weekly = [xle_weekly,xlu_weekly,xlk_weekly,xlb_weekly,xlp_weekly,xly_weekly,xli_weekly,xlc_weekly,xlv_weekly,xlf_weekly,xlre_weekly]

xle_weekly_df,xlu_weekly_df,xlk_weekly_df,xlb_weekly_df,xlp_weekly_df,xly_weekly_df,xli_weekly_df,xlc_weekly_df,xlv_weekly_df,xlf_weekly_df,xlre_weekly_df = (pd.DataFrame(x,columns=['Ticker','RSC','Price','Percent Change','Index'])
                                                                                                                                                   for x in tickers_weekly)

In [19]:
xle_monthly,xlu_monthly,xlk_monthly,xlb_monthly,xlp_monthly,xly_monthly,xli_monthly,xlc_monthly,xlv_monthly,xlf_monthly,xlre_monthly = (rsc_algo_test(x,monthly) for x in ticker_dataframe)

tickers_monthly = [xle_monthly,xlu_monthly,xlk_monthly,xlb_monthly,xlp_monthly,xly_monthly,xli_monthly,xlc_monthly,xlv_monthly,xlf_monthly,xlre_monthly]

xle_monthly_df,xlu_monthly_df,xlk_monthly_df,xlb_monthly_df,xlp_monthly_df,xly_monthly_df,xli_monthly_df,xlc_monthly_df,xlv_monthly_df,xlf_monthly_df,xlre_monthly_df = (pd.DataFrame(x,columns=['Ticker','RSC','Price','Percent Change','Index'])
                                                                                                                                                   for x in tickers_monthly)

In [20]:
xle_yearly,xlu_yearly,xlk_yearly,xlb_yearly,xlp_yearly,xly_yearly,xli_yearly,xlc_yearly,xlv_yearly,xlf_yearly,xlre_yearly = (rsc_algo_test(x,yearly) for x in ticker_dataframe)

tickers_yearly = [xle_yearly,xlu_yearly,xlk_yearly,xlb_yearly,xlp_yearly,xly_yearly,xli_yearly,xlc_yearly,xlv_yearly,xlf_yearly,xlre_yearly]

xle_yearly_df,xlu_yearly_df,xlk_yearly_df,xlb_yearly_df,xlp_yearly_df,xly_yearly_df,xli_yearly_df,xlc_yearly_df,xlv_yearly_df,xlf_yearly_df,xlre_yearly_df = (pd.DataFrame(x,columns=['Ticker','RSC','Price','Percent Change','Index'])
                                                                                                                                                   for x in tickers_yearly)

In [21]:
pd.set_option('display.max_rows', None)

In [22]:
sector_daily_df

,Ticker,RSC,Price,Percent Change,Index
0,XLK,0.005269,153.00,0.74,SPY
1,XLV,0.005257,129.98,0.74,SPY
2,XLY,0.002214,181.65,0.43,SPY
3,XLC,0.001351,81.48,0.34,SPY
4,XLU,-0.000549,64.88,0.15,SPY
5,XLB,-0.001721,81.16,0.04,SPY
6,XLP,-0.003784,70.55,-0.17,SPY
7,XLI,-0.005752,103.17,-0.37,SPY
8,XLRE,-0.009219,45.86,-0.71,SPY
9,XLF,-0.012996,36.20,-1.09,SPY


In [23]:
sector_weekly_df

,Ticker,RSC,Price,Percent Change,Index
0,XLV,0.011714,129.98,1.34,SPY
1,XLK,0.006260,153.00,0.82,SPY
2,XLY,0.006243,181.65,0.82,SPY
3,XLI,0.001894,103.17,0.41,SPY
4,XLC,-0.000274,81.48,0.16,SPY
5,XLP,-0.006728,70.55,-0.54,SPY
6,XLRE,-0.008434,45.86,-0.66,SPY
7,XLU,-0.012428,64.88,-1.09,SPY
8,XLF,-0.012881,36.20,-1.05,SPY
9,XLB,-0.015885,81.16,-1.38,SPY


In [24]:
sector_monthly_df

,Ticker,RSC,Price,Percent Change,Index
0,XLK,0.054690,153.00,9.09,SPY
1,XLY,0.023930,181.65,6.00,SPY
2,XLV,0.021145,129.98,5.70,SPY
3,XLC,-0.006355,81.48,2.98,SPY
4,XLRE,-0.015649,45.86,2.05,SPY
5,XLP,-0.027317,70.55,0.82,SPY
6,XLU,-0.036745,64.88,-0.10,SPY
7,XLI,-0.038060,103.17,-0.14,SPY
8,XLF,-0.072208,36.20,-3.51,SPY
9,XLB,-0.092189,81.16,-5.56,SPY


In [25]:
sector_yearly_df

,Ticker,RSC,Price,Percent Change,Index
0,XLF,0.116897,36.20,44.00,SPY
1,XLK,0.063691,153.00,39.62,SPY
2,XLI,0.062037,103.17,38.48,SPY
3,XLC,0.057749,81.48,38.37,SPY
4,XLE,0.046008,48.70,37.35,SPY
5,XLRE,0.002437,45.86,32.18,SPY
6,XLY,-0.008246,181.65,31.77,SPY
7,XLB,-0.011170,81.16,31.30,SPY
8,XLV,-0.091744,129.98,22.67,SPY
9,XLP,-0.156274,70.55,15.90,SPY


In [37]:
xlk_daily_df

,Ticker,RSC,Price,Percent Change,Index
0,CRM,0.017977,248.28,2.55,XLK
1,FTNT,0.015560,270.75,2.30,XLK
2,ADSK,0.014818,308.51,2.23,XLK
3,AMD,0.012664,91.21,2.01,XLK
4,ADBE,0.011195,623.68,1.87,XLK
5,ANET,0.009442,373.37,1.69,XLK
6,NOW,0.009431,576.70,1.69,XLK
7,MSFT,0.009401,286.14,1.68,XLK
8,TYL,0.009297,493.81,1.67,XLK
9,CDNS,0.008618,143.02,1.61,XLK
